In [165]:
#this file contains learning steps for building a neural network

#this is little organization to it


from platform import python_version

print(python_version())

import torch
print(torch.__version__)

import numpy as np
print(np.__version__)

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


import cv2
import os

# https://www.cs.virginia.edu/~vicente/recognition/notebooks/image_processing_lab.html
# and Yankun 
# Write the function for uploading all png files in the file folder.
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

folder = r"C:\Users\hands\Fiolic\images46"

#https://pytorch.org/tutorials/beginner/pytorch_with_examples.html

3.7.9
1.6.0
1.19.1


In [228]:
#load all the images , right now it was a folder of the first 46 images from images_001 
img = load_images_from_folder(folder)
print(len(img))

46


In [167]:
#convert the ima to a FloatTensor object
img_tensor = torch.FloatTensor(img)
print(img_tensor.shape)

46


In [188]:

# for i in img_tensor:
#     i = torch.flatten(i,start_dim=1)
#img_tensorFlat = torch.flatten(img_tensor,start_dim=1)
#print(img_tensorFlat.shape)

torch.Size([46, 1024, 1024, 3])


In [189]:
#choose first 40 for training
#choose last 6 for testing
x_train = img_tensor[:40]
x_test = img_tensor[40:]
print(x_train.shape)
print(x_test.shape)




torch.Size([40, 1024, 1024, 3])
torch.Size([6, 1024, 1024, 3])


In [190]:
#I used the first 2 colums of the Data_entry_2017.csv in a new csv called testdata

with open("testdata.csv", "r") as t:
    x = t.readlines() 



In [191]:
#import the data into a list, for lables 
#if the image had "No Finding" then a 0 was recorded, anything else was a 1 
imgLabels = []
for line in x:
    if "No Finding" in line:
        imgLabels.append([0])
    else:
        imgLabels.append([1])
        
print(imgLabels)

[[1], [1], [1], [0], [1], [1], [1], [1], [1], [1], [1], [1], [1], [0], [0], [0], [0], [0], [0], [1], [1], [0], [0], [1], [0], [1], [1], [1], [1], [0], [0], [0], [0], [1], [1], [1], [0], [1], [0], [1], [1], [1], [1], [1], [1], [1]]


In [204]:
#same as before, now for labels----- turn to tensor float then split into 
labels_tensor = torch.FloatTensor(imgLabels)
y_train = labels_tensor[:40]
y_test = labels_tensor[40:]

print(y_train.shape)
print(y_test.shape)

torch.Size([40, 1])
torch.Size([6, 1])


In [208]:
#https://towardsdatascience.com/pytorch-tabular-binary-classification-a0368da5bb89
EPOCHS = 20
BATCH_SIZE = 1
LEARNING_RATE = 0.001

class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = trainData(x_train, 
                       y_train)
## test data    
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

test_data = testData(x_test)


In [225]:
#print(train_data[0])
#combine the data for images and lables,
# make into DataLoader object
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [149]:
#first atempt at layering (do not use)
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        # 1024x1024x3 = 3145728
        self.layer_1 = nn.Linear(3145728,32) 
#         self.layer_2 = nn.Linear(1024,256)
        self.layer_out = nn.Linear(32,1) 
        
        self.relu = nn.ReLU()
#         self.dropout = nn.Dropout(p=0.1)
#         self.batchnorm1 = nn.BatchNorm1d(1024)
#         self.batchnorm2 = nn.BatchNorm1d(256)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
#         x = self.relu(self.layer_2(x))
#         x = self.batchnorm2(x)
#         x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [145]:
#figure out if GPU can be used
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [146]:
#did not work

model = binaryClassification()
model.to(device)
print(model)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
###################### OUTPUT ######################
# binaryClassification(
#   (layer_1): Linear(in_features=12, out_features=64, bias=True)
#   (layer_2): Linear(in_features=64, out_features=64, bias=True)
#   (layer_out): Linear(in_features=64, out_features=1, bias=True)
#   (relu): ReLU()
#   (dropout): Dropout(p=0.1, inplace=False)
#   (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
#   (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
# )

binaryClassification(
  (layer_1): Linear(in_features=3145728, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [147]:
#did not work
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [226]:
#did not work
model.train()
for e in range(1, EPOCHS-1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        print(X_batch.shape)
        y_pred = model(X_batch)
        
        print(y_batch.type(torch.LongTensor).flatten())
        loss = criterion(y_pred, y_batch.type(torch.LongTensor).flatten())
        acc = binary_acc(y_pred, y_batch.flatten())
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

torch.Size([1, 1024, 1024, 3])


RuntimeError: size mismatch, m1: [1048576 x 3], m2: [3145728 x 256] at ..\aten\src\TH/generic/THTensorMath.cpp:41

In [154]:
# Create the network and look at it's text representation
model = Network()
model

Network(
  (hidden): Linear(in_features=3145728, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid): Sigmoid()
  (softmax): Softmax(dim=1)
)

In [175]:
# Hyperparameters for our network
input_size = 3145728
hidden_sizes = [256]
output_size = 10
# Build a feed-forward network
model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], output_size),
                      nn.Softmax(dim=1))
print(model)

Sequential(
  (0): Linear(in_features=3145728, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
  (3): Softmax(dim=1)
)


In [156]:
from collections import OrderedDict
model = nn.Sequential(OrderedDict([
                      ('fc1', nn.Linear(input_size, hidden_sizes[0])),
                      ('relu1', nn.ReLU()),
                      ('fc2', nn.Linear(hidden_sizes[0], hidden_sizes[1])),
                      ('relu2', nn.ReLU()),
                      ('output', nn.Linear(hidden_sizes[1], output_size)),
                      ('softmax', nn.Softmax(dim=1))]))

model

Sequential(
  (fc1): Linear(in_features=3145728, out_features=128, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (relu2): ReLU()
  (output): Linear(in_features=64, out_features=10, bias=True)
  (softmax): Softmax(dim=1)
)

In [157]:
print(model[0])
print(model.fc1)

Linear(in_features=3145728, out_features=128, bias=True)
Linear(in_features=3145728, out_features=128, bias=True)


In [176]:
####FOR TESTING#########
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)),
                              ])
# Download and load the training data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [180]:
####FOR TESTING#########
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim=1))

In [221]:
print(trainloader[0][0])

TypeError: 'DataLoader' object is not subscriptable

In [183]:
####FOR TESTING#########
# Define the loss
criterion = nn.NLLLoss()
# Optimizers require the parameters to optimize and a learning rate
optimizer = optim.SGD(model.parameters(), lr=0.003)
epochs = 5
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        print(images.shape)
        print(labels.shape)
        # Flatten MNIST images into a 784 long vector
        images = images.view(images.shape[0], -1)
    
        # Training pass
        optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")

torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size

torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size

KeyboardInterrupt: 

In [184]:
print(train_loader)

In [227]:
model = nn.Sequential(nn.Linear(3145728, 256),
                      nn.ReLU(),
                      nn.Linear(256, 10),
                      nn.LogSoftmax(dim=1))
# Define the loss
criterion = nn.NLLLoss()
# Optimizers require the parameters to optimize and a learning rate
optimizer = optim.SGD(model.parameters(), lr=0.003)
epochs = 5
for e in range(epochs):
    running_loss = 0
    for images, labels in train_loader:
#         print(images.shape)
#         print(labels.shape)
        # Flatten MNIST images into long vector
        
        images = images.view(images.shape[0], -1)
#         print(images)
        # Training pass
        optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels[0].type(torch.LongTensor))
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(train_loader)}")

tensor([[ 1.,  1.,  1.,  ..., 43., 43., 43.]])
tensor([[ 69.,  69.,  69.,  ..., 139., 139., 139.]])
tensor([[187., 187., 187.,  ...,   0.,   0.,   0.]])
tensor([[2., 2., 2.,  ..., 0., 0., 0.]])
tensor([[1., 1., 1.,  ..., 0., 0., 0.]])
tensor([[2., 2., 2.,  ..., 0., 0., 0.]])
tensor([[ 1.,  1.,  1.,  ..., 41., 41., 41.]])
tensor([[  7.,   7.,   7.,  ..., 158., 158., 158.]])
tensor([[208., 208., 208.,  ...,   0.,   0.,   0.]])
tensor([[ 1.,  1.,  1.,  ..., 12., 12., 12.]])
tensor([[195., 195., 195.,  ...,   0.,   0.,   0.]])
tensor([[5., 5., 5.,  ..., 0., 0., 0.]])
tensor([[77., 77., 77.,  ...,  4.,  4.,  4.]])
tensor([[ 0.,  0.,  0.,  ..., 17., 17., 17.]])
tensor([[ 64.,  64.,  64.,  ..., 218., 218., 218.]])
tensor([[186., 186., 186.,  ...,  90.,  90.,  90.]])
tensor([[250., 250., 250.,  ...,  21.,  21.,  21.]])
tensor([[ 0.,  0.,  0.,  ..., 74., 74., 74.]])
tensor([[127., 127., 127.,  ...,   0.,   0.,   0.]])
tensor([[174., 174., 174.,  ...,   0.,   0.,   0.]])
tensor([[202., 202., 202

KeyboardInterrupt: 